In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Carregar o tokenizer e o modelo BERT pré-treinado
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Carregar o CSV com frases e sentimentos
df = pd.read_csv('sentences_with_sentiments.csv')

# Extrair as frases e os scores (sentimentos)
sentences = df['Sentence'].tolist()
scores = df['Sentiment'].tolist()

# Dividir os dados em treino e validação
train_sentences, val_sentences, train_scores, val_scores = train_test_split(
    sentences, scores, test_size=0.1, random_state=42
)

# Exemplo de saída para ver se está funcionando corretamente
print(f"Treino: {len(train_sentences)} frases, Validação: {len(val_sentences)} frases")


Treino: 215307 frases, Validação: 23924 frases


In [3]:
print(torch.cuda.is_available())  # Verifica se CUDA está habilitado
print(torch.cuda.device_count())  # Número de GPUs disponíveis


True
1


In [6]:
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
from torch.nn import MSELoss
from tqdm import tqdm  # Biblioteca para a barra de progresso


# Verificar se há GPU disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Mover o modelo para a GPU, se disponível
model.to(device)

# Garantir que todas as frases sejam strings
train_sentences = [str(sentence) for sentence in train_sentences]
val_sentences = [str(sentence) for sentence in val_sentences]

# Prepare os dados
train_inputs = tokenizer(train_sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')
val_inputs = tokenizer(val_sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')
train_labels = torch.tensor(train_scores).unsqueeze(1).to(device)  # Mover rótulos para GPU
val_labels = torch.tensor(val_scores).unsqueeze(1).to(device)      # Mover rótulos para GPU

train_data = TensorDataset(train_inputs['input_ids'].to(device), 
                           train_inputs['attention_mask'].to(device), 
                           train_labels)
val_data = TensorDataset(val_inputs['input_ids'].to(device), 
                         val_inputs['attention_mask'].to(device), 
                         val_labels)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16)

# Configuração do otimizador e loss
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = MSELoss()

# Treinamento
model.train()
num_epochs = 1  # Ajuste o número de épocas conforme necessário

for epoch in range(num_epochs):
    # Usar tqdm para mostrar a barra de progresso
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch in progress_bar:
        input_ids, attention_mask, labels = batch
        
        # Zerar gradientes
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # Pegando as previsões
        
        # Cálculo da perda
        loss = loss_fn(logits, labels)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        # Atualizar a barra de progresso com a perda
        progress_bar.set_postfix({'loss': loss.item()})
    
    print(f'Epoch {epoch+1} completed.')

# Salvar o modelo e o otimizador
save_path = 'bert_sentiment_model.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': num_epochs
}, save_path)

print(f"Modelo salvo em {save_path}")

Using device: cuda


Epoch 1/1: 100%|██████████| 13457/13457 [40:08<00:00,  5.59it/s, loss=0.00515] 


Epoch 1 completed.
Modelo salvo em bert_sentiment_model.pth


In [7]:
model.eval()
with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = outputs.logits
        print(predictions)


tensor([[0.2957],
        [0.5039],
        [0.8052],
        [0.6299],
        [0.2939],
        [0.7418],
        [0.5220],
        [0.5381],
        [0.5034],
        [0.4357],
        [0.6558],
        [0.3517],
        [0.4115],
        [0.5422],
        [0.7443],
        [0.5164]], device='cuda:0')
tensor([[0.3208],
        [0.3517],
        [0.5952],
        [0.3168],
        [0.5085],
        [0.7271],
        [0.5168],
        [0.4782],
        [0.4023],
        [0.4750],
        [0.4493],
        [0.5049],
        [0.4905],
        [0.5252],
        [0.5321],
        [0.2983]], device='cuda:0')
tensor([[0.7036],
        [0.4793],
        [0.6831],
        [0.4488],
        [0.5856],
        [0.5607],
        [0.5618],
        [0.4615],
        [0.5456],
        [0.5089],
        [0.5173],
        [0.5171],
        [0.5435],
        [0.3270],
        [0.5166],
        [0.6588]], device='cuda:0')
tensor([[0.3595],
        [0.5075],
        [0.7982],
        [0.6874],
        [0

In [9]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Carregar o modelo salvo
checkpoint = torch.load('bert_sentiment_model.pth')

# Carregar o tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Carregar o modelo e seus pesos
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)
model.load_state_dict(checkpoint['model_state_dict'])

# Colocar o modelo em modo de avaliação (desativa o dropout e outras camadas de treinamento)
model.eval()

# Mover o modelo para a GPU, se disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Função para prever a frase
def predict_sentiment(sentence):
    # Tokenizar a frase
    inputs = tokenizer(sentence, padding=True, truncation=True, max_length=128, return_tensors='pt')

    # Mover a entrada para o dispositivo (GPU ou CPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Fazer a previsão com o modelo
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extrair os logits (saída do modelo)
    logits = outputs.logits

    # Como estamos prevendo um único rótulo de sentimento (regressão), os logits já são o valor da predição
    prediction = logits.item()
    
    return prediction




C:\Users\GAMER\AppData\Local\Temp\ipykernel_11200\3976971351.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('bert_sentiment_model.pth')
Some wei

Predicted Sentiment Score: 0.8468952775001526


In [19]:
# Exemplo de uso
sentence = "i am having a really good day, and i am really happy"
predicted_score = predict_sentiment(sentence)

print(f"Predicted Sentiment Score: {predicted_score}")

Predicted Sentiment Score: 0.8102856278419495
